Cargamos los datos originales del Titanic para luego limpiarlos y preparar las variables para el modelo.
Esto es fundamental para evitar problemas con valores faltantes y para transformar variables categóricas en numéricas.


In [ ]:
import pandas as pd
from google.colab import files

# Subir archivos train.csv y test.csv desde tu equipo
uploaded = files.upload()

# Cargar los datasets en dataframes
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


Estas transformaciones aseguran que el modelo reciba solo valores numéricos y que variables cualitativas relevantes se consideren, además de manejar los valores faltantes para no perder datos importantes.


In [ ]:
# Rellenar valores nulos en 'Age' y 'Fare' con la mediana para evitar sesgos
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

# Rellenar valores nulos en 'Embarked' con la moda (valor más frecuente)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)

# Crear una nueva variable 'FamilySize' sumando 'SibSp' y 'Parch' + 1 (persona misma)
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

# Extraer el título del nombre para obtener una variable categórica relevante (ej. Mr, Mrs)
for df in [train, test]:
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    # Agrupar títulos poco comunes bajo 'Rare' para evitar ruido
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col',
                                       'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                       'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

# Mapear títulos a números para que el modelo los entienda
title_map = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4}
train['Title'] = train['Title'].map(title_map)
test['Title'] = test['Title'].map(title_map)

# Rellenar posibles valores faltantes en títulos
train['Title'].fillna(4, inplace=True)
test['Title'].fillna(4, inplace=True)

# Crear variable 'Deck' basada en la letra inicial de la cabina, mapeándola a números
train['Deck'] = train['Cabin'].astype(str).str[0]
test['Deck'] = test['Cabin'].astype(str).str[0]

deck_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7, 'n': 8}
train['Deck'] = train['Deck'].map(deck_map)
test['Deck'] = test['Deck'].map(deck_map)

# Mapear variables categóricas 'Sex' y 'Embarked' a valores numéricos
sex_map = {'male': 0, 'female': 1}
train['Sex'] = train['Sex'].map(sex_map)
test['Sex'] = test['Sex'].map(sex_map)

embarked_map = {'S': 0, 'C': 1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].map(embarked_map)

Usamos solo el dataset de entrenamiento real de Kaggle (sin usar los datos de test para entrenar). Separar una parte para validación nos ayuda a evaluar el modelo de forma justa.

In [ ]:
# Definimos las columnas que usaremos como features para entrenar el modelo
features = ['Sex', 'Age', 'Deck', 'Pclass', 'Embarked', 'Title', 'FamilySize']
X = train[features]        # Variables predictoras
y = train['Survived']      # Variable objetivo

# Dividir el dataset en datos de entrenamiento y validación (80/20)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Crear y entrenar el modelo RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predecir y evaluar
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Precisión del modelo RandomForest: {accuracy:.4f}")


In [ ]:
from xgboost import XGBClassifier

# Crear y entrenar el modelo XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# Predecir y evaluar
y_pred_xgb = xgb_model.predict(X_val)
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
print(f"Precisión del modelo con XGBoost: {accuracy_xgb:.4f}")


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# Crear el modelo base
xgb = XGBClassifier(eval_metric='logloss', random_state=42)

# Definir grilla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Ejecutar la búsqueda
grid_search.fit(X, y)

# Mostrar mejores parámetros y score
print("Mejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"Mejor precisión promedio (CV): {grid_search.best_score_:.4f}")


In [ ]:
# Predecir con el mejor modelo encontrado
X_test = test[features]
best_model = grid_search.best_estimator_
final_predictions = best_model.predict(X_test)

# Crear archivo CSV
output = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': final_predictions
})

# Guardar
output.to_csv('mi_prediccion_final_xgboost.csv', index=False)

# Descargar CSV
from google.colab import files
files.download('titanic.csv')
